In [7]:
import numpy as np
import cv2
import pandas as pd
from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
# Read the Fer dataset
filename = r'C:\Users\rahul\Downloads\fer2013.csv\fer2013.csv'
df = pd.read_csv(filename)

In [3]:
df.head

<bound method NDFrame.head of        emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 colum

In [4]:

# Split the pixels column into a numpy array of integers
df['pixels'] = df['pixels'].apply(lambda x: np.array(x.split(), dtype=int))


In [5]:
# Extract LBP features for each image
lbp_features = []
for i in range(len(df)):
    img = df['pixels'][i].reshape(48, 48)
    lbp = local_binary_pattern(img, 8, 1)
    hist, _ = np.histogram(lbp, bins=np.arange(0, 257), density=True)
    lbp_features.append(hist)

In [6]:
# Apply PCA on the LBP features
pca = PCA(n_components=50)
lbp_pca = pca.fit_transform(lbp_features)

In [8]:
# Load the trained SVM model and PCA model
# Split the dataset into train and test sets
X_train = lbp_pca[:28000]
X_test = lbp_pca[28000:]
y_train = df['emotion'][:28000]
y_test = df['emotion'][28000:]

In [9]:
# Train the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Accuracy of k-NN classifier: {:.2f}%".format(accuracy_knn * 100))

Accuracy of k-NN classifier: 32.26%


In [10]:
# Train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy of SVM classifier: {:.2f}%".format(accuracy_svm * 100))

Accuracy of SVM classifier: 24.43%


In [11]:
# Train the k-Means clustering model and use it to assign labels to the test set
kmeans = KMeans(n_clusters=7, random_state=0)
kmeans.fit(X_train)
y_pred_kmeans = kmeans.predict(X_test)

C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
# Map the k-Means cluster labels to the emotion labels
cluster_labels = [2, 3, 6, 0, 4, 5, 1]
y_pred_kmeans_mapped = [cluster_labels[label] for label in y_pred_kmeans]
accuracy_kmeans = accuracy_score(y_test, y_pred_kmeans_mapped)
print("Accuracy of k-Means clustering: {:.2f}%".format(accuracy_kmeans * 100))

Accuracy of k-Means clustering: 16.24%


### TRYING TO INCREASE ACCURACY SCORE

In [13]:
# Apply PCA on the LBP features
pca = PCA(n_components=100)
lbp_pca = pca.fit_transform(lbp_features)

In [14]:
# Split the dataset into training and testing sets
train_size = int(len(df) * 0.8)
X_train, X_test = lbp_pca[:train_size], lbp_pca[train_size:]
y_train, y_test = df['emotion'][:train_size], df['emotion'][train_size:]

In [15]:
# Train SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)
print("Accuracy of SVM classifier:", svm_acc)

Accuracy of SVM classifier: 0.24714405126776262


In [16]:
# Train k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_acc = accuracy_score(y_test, knn_pred)
print("Accuracy of k-NN classifier:", knn_acc)

Accuracy of k-NN classifier: 0.3275285594873224


In [17]:
# Apply k-means clustering
kmeans = KMeans(n_clusters=7, random_state=42)
kmeans.fit(lbp_pca)
kmeans_pred = kmeans.predict(X_test)
kmeans_acc = accuracy_score(y_test, kmeans_pred)
print("Accuracy of k-Means clustering:", kmeans_acc)

C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of k-Means clustering: 0.1390359431596545


In [19]:
from sklearn.tree import DecisionTreeClassifier
# Train a Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Evaluate the Decision Tree classifier on the testing set
y_pred = dt.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of Decision Tree classifier: {acc*100:.2f}%")

Accuracy of Decision Tree classifier: 25.95%


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [21]:
# Convert data to numpy arrays and reshape for CNN input
X = np.array(lbp_features).reshape(-1, 1, 1, 256)
y = to_categorical(df['emotion'])

In [25]:
# Build a CNN model
model = Sequential()
num_classes = 7

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [38]:
import numpy as np
import pandas as pd
import cv2
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

# Read the Fer dataset
filename = r'C:\Users\rahul\Downloads\fer2013.csv\fer2013.csv'
df = pd.read_csv(filename)

# Split the pixels column into a numpy array of integers
df['pixels'] = df['pixels'].apply(lambda x: np.array(x.split(), dtype=int))

# Convert the labels to categorical format
y = to_categorical(df['emotion'])

# Reshape the image data and normalize it
X = np.array(df['pixels'].tolist()).reshape(-1, 48, 48, 1) / 255.

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

from tqdm import tqdm

# Define the number of epochs
epochs = 50

# Train the model with progress bar
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i in tqdm(range(0, len(X_train), batch_size)):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        model.train_on_batch(X_batch, y_batch)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/50


100%|██████████| 898/898 [02:48<00:00,  5.33it/s]


Epoch 2/50


100%|██████████| 898/898 [02:35<00:00,  5.76it/s]


Epoch 3/50


...

Epoch 49/50


100%|██████████| 898/898 [05:22<00:00,  2.79it/s]


Epoch 50/50


100%|██████████| 898/898 [05:09<00:00,  2.90it/s]


Test loss: 2.029048442840576
Test accuracy: 0.5253552794456482


In [39]:
import joblib

# assuming "model" is the variable containing the trained model
filename = 'cnn.joblib'
joblib.dump(model, filename)


['cnn.joblib']